In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import gc
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files
test = '/content/drive/MyDrive/tobigs14_conference/data/v_2/test_all_meta_v2.json'
test_all_meta = pd.read_json(test)

In [5]:
train = '/content/drive/MyDrive/tobigs14_conference/data/v_2/train_all_meta_v2.json'
train_all_meta = pd.read_json(train)

In [6]:
food_token = '/content/drive/MyDrive/tobigs14_conference/data/v_2/food_token.csv'
food_token_wine = pd.read_csv(food_token)

In [7]:
import pickle



f = open("/content/drive/MyDrive/tobigs14_conference/data/v_2/dataset.pkl", "rb")

temp = pickle.load(f)


In [8]:
dataset = pd.read_pickle("/content/drive/MyDrive/tobigs14_conference/data/v_2/dataset.pkl")

In [9]:
food_token = '/content/drive/MyDrive/tobigs14_conference/data/v_2/food_token_one_hot.csv'
food_token_wine = pd.read_csv(food_token)

- 변수 정리
index : raw기준 index <br>
user_note : 리뷰내용 <br>
rating_per_user : 리뷰평점 <br>
vintage_id : 빈티지 id <br>
user_like_count : 다른 사용자의 좋아요 수 <br>
user_id : 유저 ID <br>
wine_id : 와인 Id <br>
wine_name : 와인 이름(년도 포함) <br>
url : 와인 주소 <br>
like : 1=like, 0=dislike <br>
name : 와인 이름 <br>
rating_count : 평점 수 <br>
rating_average : 평점 평균 <br>
rating_distribution : 평점 분포 ({1': 30, '2': 50 ..}) <br>
label_count : 와인 생산 수 <br>
review_count : 리뷰 수 <br>
type_id : 와인 타입 ID (1: 레드와인) <br>
body : 바디 지수 <br>
acidity : 산도 지수 <br>
alcohol : 알코올 지수 <br>
food : 어울리는 음식 리스트<br>
grapes : 포도 리스트 <br>
grapes_id : 포도 ID 리스트 <br>
grapes_count : 포도별 와인 Count <br>
grape_composition : 포도 조합 비중 <br>
rank : 랭크<br>
region_id : 지역 ID <br>
region_name : 지역 이름 <br>
country_code : 지역 코드 <br>
country_most_used_grapes_id : 국가 내 가장 많이 쓴 포도 품종 ID <br>
country_most_used_grapes_name : 국가 내 가장 많이 쓴 포도 품종 이름 <br>
country_most_used_grapes_wines_count : 국가 내 가장 많이 쓴 포도 품종 와인 갯수 <br>
winery_id : 와이너리 ID(양조장) <br> 
winery_name : 와이너리 이름 <br>
winery_ratings_count : 와이너리 평가 수<br>
winery_ratings_average : 와이너리 평가 평균 <br>
winery_labels_count : 와이너리 생산 와인 수<br>
winery_wines_count : 와이너리 생산 와인종류 수 <br>
user_follower_count : 해당 유저의 팔로워 수<br>
user_following_count : 해당 유저의 팔로잉 수<br>
user_rating_count : 해당 유저의 평가 수<br>
user_rating_sum : 해당 유저의 평가 합<br>
reviews_count : 해당 유저의 리뷰 수<br>


# wine_profile 만들기 (wine_id, vintage_id, winenery_id, type_id, body, acidity, alcohol, winery_ratings_count)
winery를 넣은 이유: 같은 양조장에서 만들면 와인이 비슷할 것이고, <br> 와이너리의 평가가 높을 수록 유명한 와인이 많을 수 있기 때문

In [18]:
len(list(train_all_meta['wine_id'].unique()))

50852

In [10]:
train_all_meta.columns

Index(['index', 'user_note', 'rating_per_user', 'vintage_id',
       'user_like_count', 'userID', 'wine_id', 'wine_name', 'url', 'like',
       'name', 'rating_count', 'rating_average', 'rating_distribution',
       'label_count', 'review_count', 'type_id', 'body', 'acidity', 'alcohol',
       'food', 'grapes', 'grapes_id', 'grapes_count', 'grape_composition',
       'rank', 'region_id', 'region_name', 'country_code',
       'country_most_used_grapes_id', 'country_most_used_grapes_name',
       'country_most_used_grapes_wines_count', 'winery_id', 'winery_name',
       'winery_ratings_count', 'winery_ratings_average', 'winery_labels_count',
       'winery_wines_count', 'user_follower_count', 'user_following_count',
       'user_rating_count', 'user_rating_sum', 'reviews_count'],
      dtype='object')

In [11]:
wine_profile=train_all_meta[['wine_id', 'vintage_id', 'winery_id', 'type_id', 'body', 'acidity', 'alcohol', 'winery_ratings_count', 'like', 'rating_average']]

In [20]:
len(wine_profile['wine_id'].unique())

50852

# wine_test token화 (one_hot_encoding 사용)

In [ ]:
train_all_meta['food'][763382]

['Pasta', 'Shellfish', 'Vegetarian', 'Appetizers and snacks', 'Lean fish']

In [ ]:
train_all_meta.head()

,index,user_note,rating_per_user,vintage_id,user_like_count,userID,wine_id,wine_name,url,like,name,rating_count,rating_average,rating_distribution,label_count,review_count,type_id,body,acidity,alcohol,food,grapes,grapes_id,grapes_count,grape_composition,rank,region_id,region_name,country_code,country_most_used_grapes_id,country_most_used_grapes_name,country_most_used_grapes_wines_count,winery_id,winery_name,winery_ratings_count,winery_ratings_average,winery_labels_count,winery_wines_count,user_follower_count,user_following_count,user_rating_count,user_rating_sum,reviews_count
0,0,Soooo good 💕,4.0,164942680,0,19484511,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,34,65,383,1453.0,323
1,1,"Belíssimo champanhe rose, bem seco mais com mu...",4.0,164942680,2,352674,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,425,915,382,1516.5,297
2,4,,4.0,164942680,0,17786617,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,46,75,412,1589.0,170
3,5,Perfekt med gratinerede østers.,4.5,164942680,0,8078038,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,13,10,412,1465.0,392
4,6,Delicious!,4.0,164942680,0,3014532,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,0,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,5,0,410,1639.5,394


In [ ]:
train_all_meta['grapes_id'].values

array([list([5, 14, 110]), list([5, 14, 110]), list([5, 14, 110]), ...,
       list([5]), list([2]), list([2])], dtype=object)

In [ ]:
train_all_meta[train_all_meta['vintage_id']==92014915]['food']

763034    [Shellfish, Vegetarian, Goat cheese]
Name: food, dtype: object

In [ ]:
df=pd.DataFrame(columns=['wine_id', 'vintage_id', 'taste'])

In [ ]:
train_all_meta['vintage_id'].unique()

array([164942680, 142514980,  48346487, ...,  29209091,   2873968,
         5033394])

In [ ]:
len(list(train_all_meta['wine_id'].unique()))

50852

In [ ]:
# wine_id와 food로 묶기
df=pd.DataFrame(columns=['wine_id', 'food'])
for i in list(train_all_meta['wine_id'].unique()):
    df=df.append(train_all_meta[train_all_meta['wine_id']==i].iloc[:, [6, 20]].drop_duplicates('wine_id'))

KeyboardInterrupt: ignored

In [ ]:
# 특수문자 제거
df['food']=df['food'].map(lambda x: str(x).lstrip('[').rstrip(']'))
df['food'].replace(r'\([^)]*\)', '', regex=True, inplace=True)
df['food'].replace(r',', '|', regex=True, inplace=True)

In [ ]:
# 원핫 인코딩
one_hot=pd.Series(df['food']).str.get_dummies()

In [ ]:
df_1=pd.merge(df, one_hot, left_index=True, right_index=True, how='left')

In [ ]:
df_1.head()

,wine_id,food,'Aperitif','Appetizers and snacks','Blue cheese','Cured Meat','Fruity desserts','Game ','Goat cheese','Lamb','Lean fish','Mature and hard cheese','Mild and soft cheese','Mushrooms','Pasta','Pork','Poultry','Rich fish ','Shellfish','Spicy food','Sweet desserts','Veal','Vegetarian','Beef','Blue cheese','Fruity desserts','Game ','Lamb','Mature and hard cheese','Pasta','Pork','Poultry','Rich fish ','Shellfish','Spicy food','Sweet desserts','Veal','Vegetarian',None
0,1141133,'Pork'| 'Rich fish '| 'Shellfish'| 'Mild and s...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
30,2532733,'Beef'| 'Lamb'| 'Game '| 'Spicy food'| 'Mature...,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36,1253802,'Beef'| 'Lamb'| 'Spicy food'| 'Mature and hard...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38,1123441,'Pork'| 'Rich fish '| 'Spicy food'| 'Sweet des...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
39,1157656,'Shellfish'| 'Vegetarian'| 'Appetizers and sna...,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
df_1.to_csv('food_token_one_hot.csv')

In [ ]:
files.download('food_token_one_hot.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
food_token_one_hot = '/content/drive/MyDrive/tobigs14_conference/data/v_2/food_token_one_hot.csv'
food_token_one_hot_wine = pd.read_csv(food_token_one_hot)

In [24]:
train_all_meta['']==food_token_one_hot_wine['wine_id'].unique()

50852

## 의미없는 코드

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df1=df_1.reset_index()

In [ ]:
df_1

,wine_id,food,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1141133,"'Pork', 'Rich fish ', 'Shellfish', 'Mild and s...",'Pork','Rich fish ','Shellfish','Mild and soft cheese',None,None,None,None,None,None,None,None,None
30,2532733,"'Beef', 'Lamb', 'Game ', 'Spicy food', 'Mature...",'Beef','Lamb','Game ','Spicy food','Mature and hard cheese','Poultry',None,None,None,None,None,None,None
36,1253802,"'Beef', 'Lamb', 'Spicy food', 'Mature and hard...",'Beef','Lamb','Spicy food','Mature and hard cheese',None,None,None,None,None,None,None,None,None
38,1123441,"'Pork', 'Rich fish ', 'Spicy food', 'Sweet des...",'Pork','Rich fish ','Spicy food','Sweet desserts','Vegetarian','Poultry',None,None,None,None,None,None,None
39,1157656,"'Shellfish', 'Vegetarian', 'Appetizers and sna...",'Shellfish','Vegetarian','Appetizers and snacks','Aperitif','Cured Meat',None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741449,1197648,None,None,None,None,None,None,None,None,None,None,None,None,None,None
747906,3140995,"'Beef', 'Lamb', 'Veal'",'Beef','Lamb','Veal',None,None,None,None,None,None,None,None,None,None
751590,4239184,"'Beef', 'Veal', 'Game ', 'Poultry'",'Beef','Veal','Game ','Poultry',None,None,None,None,None,None,None,None,None
754355,1169358,"'Beef', 'Lamb', 'Game ', 'Blue cheese'",'Beef','Lamb','Game ','Blue cheese',None,None,None,None,None,None,None,None,None


In [ ]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
token_food=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names() )

In [ ]:
token_food.drop('and', axis=1)

KeyError: ignored

In [ ]:
food_token=pd.concat([df1, token_food], axis=1)

In [ ]:
files.download('food_token.json')
files.download('food_token_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
food_token = '/content/drive/MyDrive/tobigs14_conference/data/v_2/food_token.csv'
food_token_wine = pd.read_csv(food_token)
food_token_wine=food_token_wine.drop(['Unnamed: 0', 'index','vintage_id','and'], axis=1)

In [ ]:
# food에 종류별로 라벨링한것 하나로 합치기
food_cols = list(food_token_wine.columns)[2:]
food_encoded = {x: i for i, x in enumerate(food_cols)}

In [ ]:
def get_taste(food, tastes):
  def get_all_taste(gs):
    active = [str(food_encoded[taste]) for taste, g in zip(tastes, gs) if g==1]
    if len(active) == 0:
      return '0'
    return ','.join((active))
  food_token_wine['all_food'] = [
      get_all_taste(gs) for gs in zip(*[food_token_wine[taste] for taste in tastes])]
get_taste(food_token_wine, food_cols)
# 출처: https://github.com/revathijay/Python-Notebooks/blob/master/Recommendation/MovieLens_Youtube_Recommendation_Candidate_Generation_Network.ipynb

In [ ]:
food_token_wine

,wine_id,food,aperitif,appetizers,beef,blue,cheese,cured,deer,desserts,etc,fish,food.1,fruity,game,goat,hard,lamb,lean,mature,meat,mild,mushrooms,none,pasta,pork,poultry,rich,salmon,shellfish,snacks,soft,spicy,sweet,tuna,veal,vegetarian,venison,all_food
0,1141133,"['Pork', 'Rich fish (salmon, tuna etc)', 'Shel...",0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,1,0,0,0,"4,8,9,19,23,25,26,27,29,32"
1,2532733,"['Beef', 'Lamb', 'Game (deer, venison)', 'Spic...",0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,"2,4,6,10,12,14,15,17,24,30,35"
2,1253802,"['Beef', 'Lamb', 'Spicy food', 'Mature and har...",0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,"2,4,10,14,15,17,30"
3,1123441,"['Pork', 'Rich fish (salmon, tuna etc)', 'Spic...",0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,1,0,1,0,"7,8,9,10,23,24,25,26,30,31,32,34"
4,1157656,"['Shellfish', 'Vegetarian', 'Appetizers and sn...",1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,"0,1,5,18,27,28,34"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50847,1197648,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21
50848,3140995,"['Beef', 'Lamb', 'Veal']",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"2,15,33"
50849,4239184,"['Beef', 'Veal', 'Game (deer, venison)', 'Poul...",0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,"2,6,12,24,33,35"
50850,1169358,"['Beef', 'Lamb', 'Game (deer, venison)', 'Blue...",0,0,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"2,3,4,6,12,15,35"


# user_profile 만들기 (user_id, user_follower count, user_following_count, user_rating_count, user_rating_sum)

In [ ]:
user_ids = train_all_meta['userID'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

wine_ids = train_all_meta["wine_id"].unique().tolist()
wine2wine_encoded = {x: i for i, x in enumerate(wine_ids)}
wine_encoded2wine = {i: x for i, x in enumerate(wine_ids)}

In [ ]:
train_all_meta_copy = train_all_meta.copy()


In [ ]:
train_all_meta_copy['userID'] = train_all_meta["userID"].map(user2user_encoded)
#train_all_meta_copy["wine_id"] = train_all_meta["wine_id"].map(wine2wine_encoded)

In [ ]:
train_all_meta_copy

,index,user_note,rating_per_user,vintage_id,user_like_count,userID,wine_id,wine_name,url,like,name,rating_count,rating_average,rating_distribution,label_count,review_count,type_id,body,acidity,alcohol,food,grapes,grapes_id,grapes_count,grape_composition,rank,region_id,region_name,country_code,country_most_used_grapes_id,country_most_used_grapes_name,country_most_used_grapes_wines_count,winery_id,winery_name,winery_ratings_count,winery_ratings_average,winery_labels_count,winery_wines_count,user_follower_count,user_following_count,user_rating_count,user_rating_sum,reviews_count
0,0,Soooo good 💕,4.0,164942680,0,0,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,34,65,383,1453.0,323
1,1,"Belíssimo champanhe rose, bem seco mais com mu...",4.0,164942680,2,1,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,425,915,382,1516.5,297
2,4,,4.0,164942680,0,2,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,46,75,412,1589.0,170
3,5,Perfekt med gratinerede østers.,4.5,164942680,0,3,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,1,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,13,10,412,1465.0,392
4,6,Delicious!,4.0,164942680,0,4,1141133,Prestige Rosé Brut ChampagnenN.V.,/taittinger-prestige-rose-brut-champagne/w/114...,0,Prestige Rosé Brut Champagne,5248.0,4.1,"{'1': 22, '2': 68, '3': 853, '4': 3447, '5': 858}",18046.0,1798.0,3.0,3.0,3.0,12.0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...","[Chardonnay, Pinot Noir, Pinot Meunier]","[5, 14, 110]","[510440, 483935, 30275]","{'14': 70, '5': 30}",{'country': {'description': 'Rank within Franc...,409.0,Champagne,fr,"[14, 10, 5]","[Pinot Noir, Merlot, Chardonnay]","[483935, 482768, 510440]",1285.0,Taittinger,90187.0,4.2,454478.0,39.0,5,0,410,1639.5,394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763382,984107,Delicate apple and pear,3.0,26158025,0,5226,87064,Morgadio Albariñon2014,/campante-

In [ ]:
user=(pd.DataFrame(zip(train_all_meta_copy['userID'],
                        train_all_meta_copy['wine_id'],
                        train_all_meta_copy['user_follower_count'],
                        train_all_meta_copy['user_following_count'],
                        train_all_meta_copy['user_rating_count'],
                        train_all_meta_copy['user_rating_sum'],
                        ), columns = ['userID', 'wine_id', 'follower', 'following', 'rating_count', 'rating_sum']))

In [ ]:
user

,userID,wine_id,follower,following,rating_count,rating_sum
0,0,1141133,34,65,383,1453.0
1,1,1141133,425,915,382,1516.5
2,2,1141133,46,75,412,1589.0
3,3,1141133,13,10,412,1465.0
4,4,1141133,5,0,410,1639.5
...,...,...,...,...,...,...
763382,5226,87064,52,81,259,973.0
763383,5226,63654,52,81,259,973.0
763384,5226,5602,52,81,259,973.0
763385,5226,1396664,52,81,259,973.0


In [ ]:
recent_item = user.groupby('userID').agg({
    'wine_id' : [('wine_id', 'unique')]
}).reset_index()

In [ ]:
recent_item[['userID', 'wine_id']]

,userID,wine_id
,,wine_id
0,0,"[1141133, 2532733, 1253802, 1123441, 1157656, ..."
1,1,"[1141133, 23051, 18929, 1178210, 1211816, 7643..."
2,2,"[1141133, 15238, 1244, 1445351, 1136022, 1879,..."
3,3,"[1141133, 1084276, 74298, 21585, 1136856, 7005..."
4,4,"[1141133, 79160, 1128385, 16578, 74298, 78014,..."
...,...,...
6338,6338,"[1135982, 77021, 3704, 1139434, 1852273, 17083..."
6339,6339,"[1135810, 1400619, 779647, 4895567, 2021362, 6..."
6340,6340,"[21646, 2580327, 77149, 83358, 7629904, 197097..."


In [ ]:
user_data=pd.merge(user, recent_item, how = 'left', on='userID')

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [ ]:
user_data.rename(columns = {('wine_id', 'wine_id') : 'wine_tot'}, inplace = True)

In [ ]:
user_data

,userID,wine_id,follower,following,rating_count,rating_sum,wine_tot
0,0,1141133,34,65,383,1453.0,"[1141133, 2532733, 1253802, 1123441, 1157656, ..."
1,1,1141133,425,915,382,1516.5,"[1141133, 23051, 18929, 1178210, 1211816, 7643..."
2,2,1141133,46,75,412,1589.0,"[1141133, 15238, 1244, 1445351, 1136022, 1879,..."
3,3,1141133,13,10,412,1465.0,"[1141133, 1084276, 74298, 21585, 1136856, 7005..."
4,4,1141133,5,0,410,1639.5,"[1141133, 79160, 1128385, 16578, 74298, 78014,..."
...,...,...,...,...,...,...,...
763382,5226,87064,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763383,5226,63654,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763384,5226,5602,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763385,5226,1396664,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."


# predict label 뽑기 각 user들은 비슷한 와인을 뽑는다는 가정하에 wine_tot에서 random한 와인을 추출

In [ ]:
import random
random.choice(user_data['wine_tot'][1])

1232748

In [ ]:
user_data['wine_tot'][0]

array([1141133, 2532733, 1253802, 1123441, 1157656, 1134756, 1755347,
       1368193, 2310232, 1122469, 1218926, 1997514, 1206525, 1221466,
         22849, 1715702,   76137,   76129, 2167223,   14259,   73620,
       3623354, 1221576,   74557, 1175557, 1564472, 1223089,   14640,
         26594, 1514819, 1154809, 1293225, 1387694,   90192, 1410477,
         87426, 1229949,   43604, 1745846, 2590878, 1127808, 3102815,
        734294, 1130517, 1379095,  779277,   16786, 1135215, 1174845,
         17615, 1132727,   19304, 1148479, 1268465, 1823924, 1420447,
       2395909, 1156758, 1143834,    7775,   23836,   23843, 3816645,
       1391837, 1101039, 1208738,   79266,    9220,   86290, 1245076,
         16115, 1117679, 2363901,   13546, 1184388, 1269735, 2816870,
       1276374,   23995, 1188753, 1129282,   23337, 1386686,   11001,
         77130, 1173806, 1161172, 1827355, 1751560, 1725422,    4068,
          4106, 4161223, 1789271, 4488679,   31574, 4009475,    8602,
         77869,    8

In [ ]:
user_data['wine_tot'][0]random.choice(user_data['wine_tot'][0])

2590878

In [ ]:
random.choice(user_data['wine_tot'][0])

31574

In [ ]:
user_data

,userID,wine_id,follower,following,rating_count,rating_sum,wine_tot
0,0,1141133,34,65,383,1453.0,"[1141133, 2532733, 1253802, 1123441, 1157656, ..."
1,1,1141133,425,915,382,1516.5,"[1141133, 23051, 18929, 1178210, 1211816, 7643..."
2,2,1141133,46,75,412,1589.0,"[1141133, 15238, 1244, 1445351, 1136022, 1879,..."
3,3,1141133,13,10,412,1465.0,"[1141133, 1084276, 74298, 21585, 1136856, 7005..."
4,4,1141133,5,0,410,1639.5,"[1141133, 79160, 1128385, 16578, 74298, 78014,..."
...,...,...,...,...,...,...,...
763382,5226,87064,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763383,5226,63654,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763384,5226,5602,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763385,5226,1396664,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."


In [ ]:
import random
list_predict_label=[]
for idx, val in  enumerate(user_data['wine_tot']):
    #user_data['predict_label'][idx] = random.choice(val)
    list_predict_label.append(random.choice(val))
    user_data['wine_tot'][idx]=np.delete(user_data['wine_tot'][idx], np.where(user_data['wine_tot'][idx]==list_predict_label[idx]))

    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
list_predict_label

[90192, 1191976, 4378, 69664, 7316, 1106030, 88443, 1126813, 1123564, 6090996]

In [ ]:
user_data.to_csv('user_data.csv')

In [ ]:
files.download('user_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# wine_id 기준으로 wine_profile, food_token, user_profile 통합

In [ ]:
pd.merge(wine_profile, user_data,  how = 'left', on='wine_id')

In [ ]:
user_data

,userID,wine_id,follower,following,rating_count,rating_sum,wine_tot
0,0,1141133,34,65,383,1453.0,"[1141133, 2532733, 1253802, 1123441, 1157656, ..."
1,1,1141133,425,915,382,1516.5,"[1141133, 23051, 18929, 1178210, 1211816, 7643..."
2,2,1141133,46,75,412,1589.0,"[1141133, 15238, 1244, 1445351, 1136022, 1879,..."
3,3,1141133,13,10,412,1465.0,"[1141133, 1084276, 74298, 21585, 1136856, 7005..."
4,4,1141133,5,0,410,1639.5,"[1141133, 79160, 1128385, 16578, 74298, 78014,..."
...,...,...,...,...,...,...,...
763382,5226,87064,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763383,5226,63654,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763384,5226,5602,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
763385,5226,1396664,52,81,259,973.0,"[1819992, 2219287, 1895681, 1754927, 6683, 119..."
